# Minimization: Intro To Optimization

## Problema 1:

In [3]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

## Paso 1: Instalamos las librerías necesarias

In [5]:
!pip install pulp

In [6]:
import pulp as pp

## Paso 2: Recalcamos que se trata de un ejercicio de minimización de costo

In [8]:
model = pp.LpProblem(name="diet-problem", sense=pp.LpMinimize)

## Paso 3: Detallamos variables (cantidad de tabletas de cada suplemento)

In [10]:
VegaVita = pp.LpVariable(name="VegaVita", lowBound=0, cat="Integer")
HappyHealth = pp.LpVariable(name="HappyHealth", lowBound=0, cat="Integer")

## Paso 4: Colocamos los coeficientes de costo por cada suplemento

In [12]:
Cost_VegaVita = 0.20  #20 centavos de dolar
Cost_HappyHealth = 0.30  #30 centavos de dolar

## Paso 5: Creamos la función (minimizar el costo total)

In [14]:
obj_func = Cost_VegaVita * VegaVita + Cost_HappyHealth * HappyHealth
model += obj_func

## Paso 6: Creamos las restricciones 

In [16]:
C1 = pp.LpConstraint(e=20 * VegaVita + 30 * HappyHealth, sense=pp.LpConstraintGE, rhs=60, name="VitaminCConstraint")
C2 = pp.LpConstraint(e=500 * VegaVita + 250 * HappyHealth, sense=pp.LpConstraintGE, rhs=1000, name="CalciumConstraint")
C3 = pp.LpConstraint(e=9 * VegaVita + 2 * HappyHealth, sense=pp.LpConstraintGE, rhs=18, name="IronConstraint")
C4 = pp.LpConstraint(e=2 * VegaVita + 10 * HappyHealth, sense=pp.LpConstraintGE, rhs=20, name="NiacinConstraint")
C5 = pp.LpConstraint(e=60 * VegaVita + 90 * HappyHealth, sense=pp.LpConstraintGE, rhs=360, name="MagnesiumConstraint")

## Paso 7: Añadimos las restricciones a nuestro modelo

In [18]:
model += C1
model += C2
model += C3
model += C4
model += C5

## Paso 8: Resolvemos el modelo 

In [20]:
solverToUse = pp.COIN_CMD(msg=False)
model.solve()

1

## Paso 9: Mostramos los resultados

In [22]:
Results = {
    "Model Status": pp.LpStatus[model.status],
    "Optimal Cost": pp.value(model.objective)
}
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Model Status': 'Optimal',
 'Optimal Cost': 1.2000000000000002,
 'HappyHealth': 2.0,
 'VegaVita': 3.0}

In [23]:
import pandas as pd
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

,Optimal Cost,HappyHealth,VegaVita
Model Status,,,
Optimal,1.2000000000000002,2.0,3.0


Interpretación: El costo total mínimo resulta ser de aproximadamente 1.2. Este es el gasto necesario para cumplir con los requisitos nutricionales diarios usando las cantidades enteras óptimas de Vega Vita y Happy Health. Asimismo, la cantidad óptima de tabletas de Happy Health es 2 unidades, mientras que de Vega Vita resulta ser de 3 unidades.

# Problema 2: 

In [86]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQtBRpIr6Hx1_T0zJ3_DRqsE82YUjx7ZkeEKLdA64fbjtjkmc6Ibf6ebzp6CY69D482IGpG2h9GcsC5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

## Paso 1: Definimos que se trata de un problema de minimización. En este caso queremos minimizar la cantidad de conductores

In [27]:
model2 = pp.LpProblem(name="scheduling_problem", sense=pp.LpMinimize)

## Paso 2: Declaramos las variables

En esta parte declaramos variables para representar el número de conductores que inician su turno en cada uno de los posibles horarios de inicio: 0, 4, 8, 12, 16 y 20. Tipo: entero no negativo, ya que representan personas. 

In [30]:
shift_0 = pp.LpVariable(name="shift_0", lowBound=0, cat='Integer')
shift_4 = pp.LpVariable(name="shift_4", lowBound=0, cat='Integer')
shift_8 = pp.LpVariable(name="shift_8", lowBound=0, cat='Integer')
shift_12 = pp.LpVariable(name="shift_12", lowBound=0, cat='Integer')
shift_16 = pp.LpVariable(name="shift_16", lowBound=0, cat='Integer')
shift_20 = pp.LpVariable(name="shift_20", lowBound=0, cat='Integer')

## Paso 3: Definimos la función

In [32]:
model2 += (shift_0 + shift_4 + shift_8 + shift_12 + shift_16 + shift_20, "TotalDrivers")

## Paso 4: Restricciones

Debemos de tener en cuenta que en cada bloque de 4 horas debe haber suficientes conductores activos (ya sea iniciando el turno o trabajando desde uno anterior) para cubrir la demanda

0-4  : Se necesitan al menos 4 conductores. Esto puede estar cubierto por los que inician a las 0 horas y los que empiezan a las 20 horas del día anterior

4-8  : Se necesitan al menos 8 conductores, cubiertos por lo que inician a las 0 y 4 horas

8-12 : Se necesitan al menos 10 conductores, cubiertos por los que inician a las 4 y 8 horas

12-16: Se necesitan al menos 7 conductores, cubiertos por los que inician a las 8 y 12 horas

16-20: Se necesitan al menos 12 conductores, cubiertos por los que inician a las 12 y 16 horas

20-24: Se necesitan al menos 4 conductores, cubiertos por los que inician a las 16 y 20 horas

In [41]:
C1 = pp.LpConstraint(e=shift_0 + shift_20, sense=pp.LpConstraintGE, rhs=4, name="Demanda_0-4")
C2 = pp.LpConstraint(e=shift_0 + shift_4, sense=pp.LpConstraintGE, rhs=8, name="Demanda_4-8")
C3 = pp.LpConstraint(e=shift_4 + shift_8, sense=pp.LpConstraintGE, rhs=10, name="Demanda_8-12")
C4 = pp.LpConstraint(e=shift_8 + shift_12, sense=pp.LpConstraintGE, rhs=7, name="Demanda_12-16")
C5 = pp.LpConstraint(e=shift_12 + shift_16, sense=pp.LpConstraintGE, rhs=12, name="Demanda_16-20")
C6 = pp.LpConstraint(e=shift_16 + shift_20, sense=pp.LpConstraintGE, rhs=4, name="Demanda_20-24")

model2 += C1
model2 += C2
model2 += C3
model2 += C4
model2 += C5
model2 += C6

## Paso 5: Resolver el modelo

In [66]:
solver = pp.COIN_CMD(msg=False)
model2.solve()

1

## Paso 6: Los resultados

In [69]:
results = {
    "Model Status": pp.LpStatus[model2.status],
    "Total Drivers Needed": pp.value(model2.objective)
}
results.update({v.name: v.varValue for v in model2.variables()})

# Mostrar resultados en un DataFrame (opcional)
import pandas as pd
pd.DataFrame.from_dict(results, orient='index').T.set_index('Model Status').style.format('{:,}')

,Total Drivers Needed,shift_0,shift_12,shift_16,shift_20,shift_4,shift_8
Model Status,,,,,,,
Optimal,26.0,0.0,12.0,0.0,4.0,10.0,0.0


In [81]:
#Lo ordenamos
df = pd.DataFrame.from_dict(results, orient='index').T

# Reordenar las columnas 
df = df[['Model Status', 'Total Drivers Needed', 'shift_0', 'shift_4', 'shift_8', 'shift_12', 'shift_16', 'shift_20']]
df.set_index('Model Status').style.format('{:,}')

,Total Drivers Needed,shift_0,shift_4,shift_8,shift_12,shift_16,shift_20
Model Status,,,,,,,
Optimal,26.0,0.0,10.0,0.0,12.0,0.0,4.0


En conclusión: 
Se necesitan 26 conductores en total para cubrir la demanda. La asignación óptima es:
10 conductores comienzan a las 4:00 am.
12 conductores comienzan a las 12:00 pm.
4 conductores comienzan a las 8:00 pm.